In [ ]:
import osmium
import shapely.wkb as wkblib
import osmnx as ox
import geopandas

In [ ]:
#load any country you need -- exchange "Sweden" for some other country at your own risk!
!wget https://download.geofabrik.de/europe/sweden-latest.osm.pbf

In [ ]:
#you will need docker for this cell, but you can also skip this, it makes loading the osm-file faster
!docker run -d -w /wkd -v $(pwd):/wkd stefda/osmium-tool osmium tags-filter -o sweden-filtered.osm.pbf sweden-latest.osm.pbf building

In [ ]:
class BuildingHandler(osmium.SimpleHandler):
    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.nodes_count = 0
        self.nodes = []
        self.building_count = 0
        self.buildings = []
        # A global factory that creates WKB from a osmium geometry
        self.wkbfab = osmium.geom.WKBFactory()

    def node(self, n):
        if n.tags.get("building") == 'yes':
            try:
                wkb = self.wkbfab.create_point(n)
                geo = wkblib.loads(wkb, hex=True)
            except Exception as e:
                print(e)
                return
            row = { "w_id": n.id, "geometry": geo }
            
            for key, value in n.tags:
                row[key] = value
                
            self.nodes.append(row)
            self.nodes_count += 1
        
    def area(self, w):
        if w.tags.get("building") == 'yes':
            try:
                wkb = self.wkbfab.create_multipolygon(w)
                geo = wkblib.loads(wkb, hex=True)
            except Exception as e:
                print(e)
                return
            row = { "w_id": w.id, "geometry": geo }

            for key, value in w.tags:
                row[key] = value

            self.buildings.append(row)
            self.building_count += 1

In [ ]:
#loading the file into
buildinghandler = BuildingHandler()
buildinghandler.apply_file("sweden-filtered.osm.pbf", locations=True)

In [ ]:
i = 200000
while i-200000 < len(buildinghandler.buildings):
    dfx = pd.DataFrame(buildinghandler.buildings[(i-200000):min([i, len(buildinghandler.buildings)-1])])
    gdfx = geopandas.GeoDataFrame(dfx, geometry='geometry')
    gdfx = gdfx.set_crs("EPSG:4326")
    gdfx = ox.project_gdf(gdfx)
    #gdfx = gdfx.dropna(subset=['building:levels'])
    gdfx = gdfx[['w_id', 'geometry', 'building:levels']]
    if i < 200001:
        meta = gdfx
    else:
        meta = pd.concat([meta, gdfx])
    print(meta.shape)
    i += 200000

In [ ]:
reg = meta[meta['building:levels'].str.contains('[A-Za-z]') == False]
reg = reg[reg['building:levels'].str.contains('[;-]') == False]

In [ ]:
#use method .area to get area of buildings, e.g., reg.area